In [34]:
import numpy as np
import pandas as pd
import re
import jieba
from multiprocessing import cpu_count, Pool

In [3]:
train_df = pd.read_csv('data/AutoMaster_TrainSet.csv')
test_df = pd.read_csv('data/AutoMaster_TestSet.csv')

train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82943 entries, 0 to 82942
Data columns (total 6 columns):
QID         82943 non-null object
Brand       81642 non-null object
Model       81642 non-null object
Question    82943 non-null object
Dialogue    82941 non-null object
Report      82873 non-null object
dtypes: object(6)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
QID         20000 non-null object
Brand       19987 non-null object
Model       19987 non-null object
Question    20000 non-null object
Dialogue    20000 non-null object
dtypes: object(5)
memory usage: 781.4+ KB


In [4]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


## 1.去除NA值

In [9]:
train_df.dropna(subset=['Question','Dialogue','Report'], how='any', inplace=True)
test_df.dropna(subset=['Question','Dialogue'], how='any', inplace=True)

train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82871 entries, 0 to 82942
Data columns (total 6 columns):
QID         82871 non-null object
Brand       81572 non-null object
Model       81572 non-null object
Question    82871 non-null object
Dialogue    82871 non-null object
Report      82871 non-null object
dtypes: object(6)
memory usage: 4.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 5 columns):
QID         20000 non-null object
Brand       19987 non-null object
Model       19987 non-null object
Question    20000 non-null object
Dialogue    20000 non-null object
dtypes: object(5)
memory usage: 937.5+ KB


## 2. 过滤特殊字符 

In [11]:
def clean_sentence(sentence):
    '''
    特殊字符去除
    :param sentence:待处理字符串
    :return:过滤之后字符串
    '''
    if isinstance(sentence, str):
        return re.sub(
            r'[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】“”！，。？、~@#￥%……&*（）]+|车主说|技师说|语音|图片|你好|您好',
            '', sentence)
    else:
        return ''

In [14]:
clean_sentence(train_df.Dialogue[2])

'4缸自然吸气发动机N46是吧先挂空档再挂其他档有没有闯动呢变速箱油液位是否调整到正常液位呢从N到D进本没有NR有PR有最主要是行驶中到红绿灯路口等红灯停车DN有冲击感绿灯后ND冲击感很小第一要把变速箱油位调整到标准液位第二清除变速箱适应值第三升级变速箱程序还有遇到过液力变矩器问题的升级变速箱程序是刷模块吗还有个问题就是停车后档位在P挡松开刹车踏板时感觉车辆会动一下怎样清除变速箱适应值先从简单的排查吧有可能程序问题也有可能液力变矩器轴头磨损泄压了需要专用电脑清除变速箱适应值升级变速箱程序换变速箱油有焦糊味没变速箱油底壳带滤芯的换了没没有味滤芯换了变矩器磨损的话车况上架4轮离地换挡位就没有冲击感呢对所以先从简单的排查换了油也需要也需要重新学习没换油之前就这样是因为有冲击才换的油换油之前也是换挡冲击还是行驶也有冲击只是原地换挡位冲击换油前后行驶都没问题多少公里了估计还是程序问题阀体里的问题阀体和电脑一体的93万公里我昨天去试了一个4万多公里X1也是这样是不是通病如果是有一点那是正常的刹车踩重点也是吗用力踩刹车的话冲击感基本没有就是用力踩住刹车原地换挡位基本感觉不到冲击感如果是这样行驶没有冲击应该没有问题哦哦谢谢不客气'

## 3. jieba分词
+ 加载汽车词典
+ 过滤停用词

In [19]:
print(list(jieba.cut(clean_sentence(train_df.Dialogue[2]))))

['4', '缸', '自然', '吸气', '发动机', 'N46', '是', '吧', '先', '挂', '空档', '再', '挂', '其他', '档', '有没有', '闯动', '呢', '变速箱', '油液', '位', '是否', '调整', '到', '正常', '液位', '呢', '从', 'N', '到', 'D', '进本', '没有', 'NR', '有', 'PR', '有', '最', '主要', '是', '行驶', '中到', '红绿灯', '路口', '等', '红灯', '停车', 'DN', '有', '冲击', '感', '绿灯', '后', 'ND', '冲击', '感', '很小', '第一', '要', '把', '变速箱', '油位', '调整', '到', '标准', '液位', '第二', '清除', '变速箱', '适应', '值', '第三', '升级', '变速箱', '程序', '还有', '遇到', '过', '液力', '变矩器', '问题', '的', '升级', '变速箱', '程序', '是', '刷', '模块', '吗', '还有', '个', '问题', '就是', '停车', '后', '档位', '在', 'P', '挡', '松开', '刹车踏板', '时', '感觉', '车辆', '会', '动', '一下', '怎样', '清除', '变速箱', '适应', '值先', '从', '简单', '的', '排查', '吧', '有', '可能', '程序', '问题', '也', '有', '可能', '液力', '变矩器', '轴头', '磨损', '泄压', '了', '需要', '专用', '电脑', '清除', '变速箱', '适应', '值', '升级', '变速箱', '程序', '换', '变速箱', '油有', '焦糊', '味', '没', '变速箱', '油', '底壳', '带', '滤芯', '的', '换', '了', '没', '没有', '味', '滤芯', '换', '了', '变矩器', '磨损', '的话', '车况', '上架', '4', '轮离', '地', '换挡', '位', '就', '没有', '冲击', '感', '呢',

In [20]:
# 加载汽车词典
jieba.load_userdict('data/car_dict.txt')
print(list(jieba.cut(clean_sentence(train_df.Dialogue[2]))))

['4', '缸', '自然', '吸气', '发动机', 'N46', '是', '吧', '先', '挂', '空档', '再', '挂', '其他', '档', '有没有', '闯动', '呢', '变速箱', '油液', '位', '是否', '调整', '到', '正常', '液位', '呢', '从', 'N', '到', 'D', '进本', '没有', 'NR', '有', 'PR', '有', '最', '主要', '是', '行驶', '中到', '红绿灯', '路口', '等', '红灯', '停车', 'DN', '有', '冲击', '感', '绿灯', '后', 'ND', '冲击', '感', '很小', '第一', '要', '把', '变速箱', '油位', '调整', '到', '标准', '液位', '第二', '清除', '变速箱', '适应', '值', '第三', '升级', '变速箱', '程序', '还有', '遇到', '过', '液力', '变矩器', '问题', '的', '升级', '变速箱', '程序', '是', '刷', '模块', '吗', '还有', '个', '问题', '就是', '停车', '后', '档位', '在', 'P', '挡', '松开', '刹车踏板', '时', '感觉', '车辆', '会', '动', '一下', '怎样', '清除', '变速箱', '适应', '值先', '从', '简单', '的', '排查', '吧', '有', '可能', '程序', '问题', '也', '有', '可能', '液力', '变矩器', '轴头', '磨损', '泄压', '了', '需要', '专用', '电脑', '清除', '变速箱', '适应', '值', '升级', '变速箱', '程序', '换', '变速箱', '油有', '焦糊', '味', '没', '变速箱油底壳', '带', '滤芯', '的', '换', '了', '没', '没有', '味', '滤芯', '换', '了', '变矩器', '磨损', '的话', '车况', '上架', '4', '轮离', '地', '换挡', '位', '就', '没有', '冲击', '感', '呢', '对', '所

In [21]:
# 加载停用词
def load_stop_words(stop_word_path):
    '''
    加载停用词词典
    :param stop_word_path:停用词路径
    :return 停用词list
    '''
    with open(stop_word_path, encoding='utf8') as f:
        stop_words = f.readlines()
        stop_words = [x.strip() for x in stop_words]
    return stop_words

In [22]:
stop_words = load_stop_words('stopwords/哈工大停用词表.txt')
len(stop_words)

767

In [23]:
def filter_stopwords(words):
    '''
    过滤停用词（已加载停用词到stopwords）
    :param words: 待处理语句list
    :return: 过滤后停用词
    '''
    return [x for x in words if x not in stop_words]

In [25]:
print(filter_stopwords(list(jieba.cut(clean_sentence(train_df.Dialogue[2])))))

['4', '缸', '自然', '吸气', '发动机', 'N46', '先', '挂', '空档', '再', '挂', '档', '有没有', '闯动', '变速箱', '油液', '位', '是否', '调整', '正常', '液位', 'N', 'D', '进本', '没有', 'NR', 'PR', '最', '主要', '行驶', '中到', '红绿灯', '路口', '红灯', '停车', 'DN', '冲击', '感', '绿灯', '后', 'ND', '冲击', '感', '很小', '第一', '变速箱', '油位', '调整', '标准', '液位', '清除', '变速箱', '适应', '值', '第三', '升级', '变速箱', '程序', '遇到', '液力', '变矩器', '问题', '升级', '变速箱', '程序', '刷', '模块', '问题', '停车', '后', '档位', 'P', '挡', '松开', '刹车踏板', '时', '感觉', '车辆', '会', '动', '一下', '清除', '变速箱', '适应', '值先', '简单', '排查', '可能', '程序', '问题', '可能', '液力', '变矩器', '轴头', '磨损', '泄压', '需要', '专用', '电脑', '清除', '变速箱', '适应', '值', '升级', '变速箱', '程序', '换', '变速箱', '油有', '焦糊', '味', '没', '变速箱油底壳', '带', '滤芯', '换', '没', '没有', '味', '滤芯', '换', '变矩器', '磨损', '车况', '上架', '4', '轮离', '换挡', '位', '没有', '冲击', '感', '先', '简单', '排查', '换', '油', '需要', '需要', '重新学习', '没', '换油', '之前', '是因为', '冲击', '才', '换', '油', '换油', '之前', '换挡', '冲击', '行驶', '冲击', '原地', '换挡', '位', '冲击', '换油', '行驶', '都', '没', '问题', '公里', '估计', '程序', '问题', '阀体', '里', '问题',

## 4.拼接流程，批处理

In [31]:
def process_sentence(sentence):
    '''
    预处理流程
    :param sentence:待处理字符串
    :return 处理后字符串
    '''
    # 清除无用词
    sentence = clean_sentence(sentence)
    # 分词
    words = jieba.cut(sentence)
    # 过滤停用词
    words = filter_stopwords(words)
    # 以空格连接词组
    return ' '.join(words)

In [27]:
process_sentence(train_df.Dialogue[2])

'4 缸 自然 吸气 发动机 N46 先 挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位 是否 调整 正常 液位 N D 进本 没有 NR PR 最 主要 行驶 中到 红绿灯 路口 红灯 停车 DN 冲击 感 绿灯 后 ND 冲击 感 很小 第一 变速箱 油位 调整 标准 液位 清除 变速箱 适应 值 第三 升级 变速箱 程序 遇到 液力 变矩器 问题 升级 变速箱 程序 刷 模块 问题 停车 后 档位 P 挡 松开 刹车踏板 时 感觉 车辆 会 动 一下 清除 变速箱 适应 值先 简单 排查 可能 程序 问题 可能 液力 变矩器 轴头 磨损 泄压 需要 专用 电脑 清除 变速箱 适应 值 升级 变速箱 程序 换 变速箱 油有 焦糊 味 没 变速箱油底壳 带 滤芯 换 没 没有 味 滤芯 换 变矩器 磨损 车况 上架 4 轮离 换挡 位 没有 冲击 感 先 简单 排查 换 油 需要 需要 重新学习 没 换油 之前 是因为 冲击 才 换 油 换油 之前 换挡 冲击 行驶 冲击 原地 换挡 位 冲击 换油 行驶 都 没 问题 公里 估计 程序 问题 阀体 里 问题 阀体 电脑 一体 93 万公里 昨天 去试 4 万多公里 X1 是不是 通病 一点 正常 刹车 踩 重点 用力 踩 刹车 冲击 感 基本 没有 用力 踩住 刹车 原地 换挡 位 基本 感觉 不到 冲击 感 行驶 没有 冲击 应该 没有 问题 谢谢 不 客气'

In [32]:
# 批处理
def process_dataframe(df):
    '''
    数据集批量处理方法
    :param df: 数据集
    :return:处理好的数据集
    '''
    # 批量预处理 训练集和测试集
    for col_name in ['Brand', 'Model', 'Question', 'Dialogue']:
        df[col_name] = df[col_name].apply(process_sentence)

    if 'Report' in df.columns:
        # 训练集 Report 预处理
        df['Report'] = df['Report'].apply(process_sentence)
    return df

In [33]:
%%time
process_dataframe(train_df)
process_dataframe(test_df)

CPU times: user 4min 16s, sys: 914 ms, total: 4min 17s
Wall time: 4min 18s


,QID,Brand,Model,Question,Dialogue
0,Q1,大众 进口,高尔夫 进口,帕萨特 烧 机油,请问 车 跑 公里 保修期 内 当地 4 店 里面 进行 检查 维修 已经 超出 保修期 建...
1,Q2,一汽大众 奥迪,奥迪 A6,修 一下 钱 换 修,师傅 抛光 处理 一下 好 50 元 左右 好 希望 能够 帮到 祝 生活 愉快
2,Q3,上汽 大众,帕萨特,帕萨特 领域 喇叭 坏 店里 说 方向盘 里线 坏 换 一根 两三百 不 感觉 太贵,气囊 油丝坏 价格 不 贵 更换
3,Q4,南京菲亚特,派力奥,发动机 漏气 会 征兆,发动机 没力 伴有 啪啪 漏气 声音 二 发动机 没力 伴有 排气管 黑烟 三 水温 高水箱...
4,Q5,东风本田,思铂睿,请问 那天 右后 胎扎 订补 胎后 跑 高速 80 有点 抖 110 时速 以上 抖动 明显...,师傅 可能 前轮 平衡 快 脱落 不 平衡 造成 建议 前轮 做 一下 动平衡 好 希望 能...
...,...,...,...,...,...
19995,Q19996,路虎,揽胜 极光,路虎 极光 20t 发动机 进气 链轮 损坏 更换 进气 链轮 后 发动 一会儿 自动 熄火...,主要 检查 正时 专用工具 专用工具 两大 疑问 旧 链轮 转动 时 发现 进气凸轮轴 明显...
19996,Q19997,别克,英朗,别克英朗 XT 别撞 后 发现 左 雾灯 下雨 后 起雾 更换 新 原厂 雾灯 需要 钱 买,原厂 件 厂家 合作 汽配 厂 生产 都 品牌 想要 完全 只能 服务站 订货 汽配城 发 ...
19997,Q19998,大众,朗逸,师傅 想 问 下车 一周 开 两次 周一 开 过来 周五 开回去 对车 影响,车 不要 经常 停放 超过 一周 需要 电瓶 负极 线 拆掉 避免 电瓶 漏电 亏损 没事 ...
19998,Q19999,金杯,雷龙,大师 车前 分泵 制动钳 导管 之间 晃动 颠簸 路段 当当响 应该 处理,最 简单 办法 中间 垫点 薄 铁皮 祝您 用车 愉快 导管 踩 刹车 时要 活动 里面 垫...


## 5.多核运行

In [36]:
?np.array_split

Signature: np.array_split(ary, indices_or_sections, axis=0)
Docstring:
Split an array into multiple sub-arrays.

Please refer to the ``split`` documentation.  The only difference
between these functions is that ``array_split`` allows
`indices_or_sections` to be an integer that does *not* equally
divide the axis. For an array of length l that should be split
into n sections, it returns l % n sub-arrays of size l//n + 1
and the rest of size l//n.

See Also
--------
split : Split array into multiple sub-arrays of equal size.

Examples
--------
>>> x = np.arange(8.0)
>>> np.array_split(x, 3)
    [array([0.,  1.,  2.]), array([3.,  4.,  5.]), array([6.,  7.])]

>>> x = np.arange(7.0)
>>> np.array_split(x, 3)
    [array([0.,  1.,  2.]), array([3.,  4.]), array([5.,  6.])]
File:      ~/opt/miniconda3/envs/kaikeba/lib/python3.6/site-packages/numpy/lib/shape_base.py
Type:      function


In [37]:
def parallelize(df, func):
    '''
    多核运行func程序处理df
    :param df: 待处理dataframe
    :param func: 批处理流程
    :return: 处理后df
    '''
    # cpu数量
    cores = cpu_count()
    # 分块数量
    partitions = cores
    # split data
    data_split = np.array_split(df, partitions)
    # open process pool
    pool = Pool()
    data = pd.concat(pool.map(func, data_split))
    # close process pool
    pool.close()
    # 执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    pool.join()
    return data

In [38]:
%%time
train_df = parallelize(train_df, process_dataframe)
test_df = parallelize(test_df, process_dataframe)

CPU times: user 1.03 s, sys: 543 ms, total: 1.57 s
Wall time: 1min 5s


In [39]:
cores

8

In [40]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰 GL 级,方向机 重 助力泵 方向机 都 换,新 都 换 助力泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问题...,随时 联系
1,Q2,奔驰,奔驰 M 级,奔驰 ML500 排气凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马 X1 进口,2010 款 宝马 X12011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位 ...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰 C 级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场 价钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


## 6.保存数据
+ QA问题，提取Question、Dialogue、Report作为vocab

In [41]:
train_df['merge'] = train_df[['Question','Dialogue','Report']].apply(lambda x : ' '.join(x), axis=1)
test_df['merge'] = test_df[['Question','Dialogue']].apply(lambda x : ' '.join(x), axis=1)

In [44]:
merged_df = pd.concat([train_df['merge'], test_df['merge']], axis=0)
print('train data size {},test data size {},merged_df data size {}'.format(len(train_df),len(test_df),len(merged_df)))

train data size 82871,test data size 20000,merged_df data size 102871


In [45]:
merged_df.to_csv('data/merged_train_test_seg_data.csv', index=None, header=False)

## 7.构建vocab

In [49]:
vocab = set(' '.join(merged_df).split(' '))
len(vocab)

160656

In [56]:
import os
import pickle
def save_file(data, filepath):
    dirs = filepath.split(os.sep)[:-1]
    DIR = '.'
    while len(dirs):
        DIR += os.sep + dirs.pop(0)
        if not os.path.isdir(DIR):
            os.mkdir(DIR)
    if not filepath.endswith('.pkl'):
        filepath += '.pkl'
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

In [57]:
save_file(vocab, 'data/vocab')

In [ ]:
def pa